<h2><center>IIC2440 – Procesamiento de Datos Masivos</center></h2>
<h3><center>Tarea 1: Análisis sesiones parlamentarias</center></h3>


**Integrantes:**
- Franco Chiappe
- Tomás Pérez

### Librerias

In [274]:
# Importar librerias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import unicodedata

## Parte 1 [1 pto] - Procesando los datos CREO QUE LISTO

In [275]:
# Cargar los datos
parlamentarios_info = pd.read_csv('participacion/parlamentarios_info_general.csv')
descripcion_2023_1 = pd.read_csv('participacion/participacion_descripcion_2023_1.csv')
descripcion_2023_2 = pd.read_csv('participacion/participacion_descripcion_2023_2.csv')
descripcion_2024 = pd.read_csv('participacion/participacion_descripcion_2024.csv')
df = pd.concat([descripcion_2023_1, descripcion_2023_2, descripcion_2024], ignore_index=True)

### Pre-Procesamiento de Datos

In [276]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68161 entries, 0 to 68160
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    68161 non-null  int64  
 1   PARLAMENTARIO_ID    68161 non-null  int64  
 2   FECHA               68161 non-null  object 
 3   LEGISLATURA         68161 non-null  int64  
 4   SESION              68161 non-null  int64  
 5   TIPO_SESION         68161 non-null  object 
 6   TIPO_PARTICIPACION  68161 non-null  object 
 7   CAMARA              68161 non-null  object 
 8   PARLAMENTARIOS      68161 non-null  object 
 9   TEXTO_PRINCIPAL     68161 non-null  object 
 10  TEXTO_ANTECEDENTE   32393 non-null  object 
 11  DESCRIPCION_DEBATE  68161 non-null  object 
 12  BOLETIN_ID          20593 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 6.8+ MB


In [277]:
# Revisión Datos Nulos:
df.isna().sum()

ID_PARTICIPACION          0
PARLAMENTARIO_ID          0
FECHA                     0
LEGISLATURA               0
SESION                    0
                      ...  
PARLAMENTARIOS            0
TEXTO_PRINCIPAL           0
TEXTO_ANTECEDENTE     35768
DESCRIPCION_DEBATE        0
BOLETIN_ID            47568
Length: 13, dtype: int64

Nos damos cuenta que en las columnas:

- `TEXTO_ANTECEDENTE` tiene 35768  datos nulos
- `BOLETIN_ID` tiene 47568 datos nulos

Decidimos eliminar la columna `TEXTO_ANTECEDENTE`, porque presentaba un alto porcentaje de datos nulos y no aportaba informacion relevante. Por otro parte, no eliminamos la columna `BOLETIN_ID`, esta columna podria resultar util para analisis futuros.

In [278]:
df = df.drop(columns=["TEXTO_ANTECEDENTE"])

### Limpieza de texto

In [279]:
def limpiar_texto(texto):
        
    texto = texto.lower()
    texto = texto.strip()
    
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

    return texto

Se usa la funcion ***limpiar_texto*** el objetivo es normalizar los textos para facilitar la extraccion de keywords. Esta funcion convierte todo el contenido a minuscula, elimina tildes, signos de puntuación y espacios innecesarios, dejando un text limpio. Esto permite detectar con mayor precision los temas.

In [280]:
# Aplicar la funcion de limpieza a las columnas
df['DESCRIPCION_DEBATE'] = df['DESCRIPCION_DEBATE'].apply(limpiar_texto)
df['TEXTO_PRINCIPAL'] = df['TEXTO_PRINCIPAL'].apply(limpiar_texto)

# Juntamos las columnas
df["TEXTO_COMBINADO"] = df["TEXTO_PRINCIPAL"] + " " + df["DESCRIPCION_DEBATE"]
df["TEXTO_COMBINADO"].head()

0    la senora allende ngracias presidente npor su ...
1    la senora allende ngracias presidente nen prim...
2    proyecto de acuerdo de los honorables senadore...
3    de la senora allende n a la municipalidad de v...
4    proyecto de acuerdo de los honorables senadore...
Name: TEXTO_COMBINADO, dtype: object

### Temas y palabras claves

In [281]:
temas = {
    "salud": ["salud", "hospital", "hospitales" "medico", "vacuna", "enfermedad", "medicina", "medicamento"],
    "educacion": ["educacion", "colegio", "escuela", "universidad", "profesor", "alumno", "estudiante", "educador", "colegios"],
    "delincuencia": ["delincuencia", "carabinero", "robo", "narcotrafico", "seguridad", "delito", "delincuente", "delincuentes"],
    "medio ambiente": ["medioambiente", "naturaleza", "contaminacion", "reciclaje", "agua", "bosque", "bosques", "flora", "fauna", "ecosistema", "ecosistemas"],
    "economia": ["economia", "dinero", "impuesto", "trabajo", "impuestos", "empresa", "empresas", "negocio", "negocios", "inversion", "inversiones", "financiero", "financieros"],
    "constitucion": ["constitucion", "constitucional"],
    "vivienda": ["vivienda", "casa", "alquiler", "arrendar"],
    "transporte": ["transporte", "metro", "camino", "micro"],
    "derechos humanos": ["derechos humanos", "derechos", "humanos", "discriminacion", "discriminacion", "discriminados", "discriminados", "discriminacion", "discriminaciones", "humanitaria"],
    "legislacion": ["legislacion", "ley", "leyes", "legislativo", "legislativa"],
    "justicia": ["justicia", "justicia", "jueces", "juez", "tribunal", "tribunales"],
}

<h1 style="color:red;"><center> Que otras palabras podemos incluir? </center></h1>

In [282]:
def asignar_keywords(texto):
    palabras = texto.split()
    etiquetas = set()
    for tema, claves in temas.items():
        for palabra in claves:
            if (palabra in palabras):
                etiquetas.add(tema)
       
    return list(etiquetas)

In [283]:
# Se agrupan por descripción debate, para poder aplicar la función de asignar_keywords a cada grupo de manera más eficiente
grouped_df = df.groupby("DESCRIPCION_DEBATE").first().reset_index()
grouped_df["KEYWORDS"] = grouped_df["TEXTO_COMBINADO"].apply(asignar_keywords)

df = df.merge(grouped_df[["TEXTO_COMBINADO", "KEYWORDS"]], on="TEXTO_COMBINADO", how="left")
df["KEYWORDS"].value_counts()

KEYWORDS
[]                                                                          4836
[legislacion]                                                               4060
[economia]                                                                   814
[salud]                                                                      672
[economia, legislacion]                                                      650
                                                                            ... 
[salud, justicia, legislacion, delincuencia]                                   1
[justicia, economia, derechos humanos, salud, medio ambiente]                  1
[educacion, justicia, economia, legislacion, transporte]                       1
[vivienda, economia, derechos humanos, legislacion, salud, constitucion]       1
[medio ambiente, derechos humanos, justicia, transporte]                       1
Name: count, Length: 524, dtype: int64

### Conclusion:
Se realizo el preprocesamiento de las intervenciones parlamentarias con el objetivo de identificar las tematicas. Para esto, se aplico una funcion de limpieza que normalizo el texto (conversion a minusculas, eliminacion de tildes, puntuacion y espacios innecesarios). Luego, se unieron las columnas `TEXTO_ANTECEDENTE` y `DESCRIPCION_DEBATE` para generar un texto combinado. Finalmente, se aplico una funcion que detecta las palabras clave y asigna a cada intervencion una o mas temas, permitiendo categorizar los discursos parlamentarios segun temas como salud, educación, delincuencia, entre otros. Para optimizar la asignación, se agruparon las filas por la columna `DESCRIPCION_DEBATE`, ya que muchas compartian o eran de un mismo debate, para así acortar el tiempo de ejecución de asignación

In [284]:
df["KEYWORDS"].explode().value_counts()

KEYWORDS
legislacion         9216
economia            3444
delincuencia        2421
constitucion        2354
salud               1880
                    ... 
derechos humanos    1724
justicia            1172
transporte          1006
medio ambiente      1002
vivienda             689
Name: count, Length: 11, dtype: int64

## Parte 2 - [1 pts] - Modelamiento

## Parte 3 - [1.5 pts] - BigQuery

## Parte 4 - [2 pts] - Informe>

## Parte 5 - [0.5 pts] - Resumen ejecutivo

Hay que separar los 5932, ver si se va a hacer con keywords 

Esto no sirve directamente, pero es para saber que hay varios que tienen detalles así y que al revisarlo por texto sale similiar